# 第七章：LangChain 与 LangGraph 的基础知识

 Agent 最重要的两个道路：
  - Prompt Engineering Based Agent System
  - RL强化学习优化单个模型性能上线

## 1. 理解LangChain Basic

In [ ]:
! pip install langchain

In [3]:
from dotenv import load_dotenv
import os

load_dotenv('../../../.env')  # 路径在上面三层

True

In [6]:
BASE_URL = "https://api.deepseek.com"
API_KEY = os.getenv('DEEPSEEK_API_KEY')
if not API_KEY:
    raise ValueError("API key not found. Please set the DEEPSEEK_API_KEY environment variable.")

deep_seek_chat_model = 'deepseek-chat'
deep_seek_reason_model = 'deepseek-reasoner'

### 作者使用LangChain和LangGraph的经验
- langchain
  - init model
  - invoke
  - prompt template
- Langgraph
  - memory
  - create_react_agent

下面分享三种langchain中初始化模型的方法

###  1.1-方式1：openai sdk

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
    model_name=deep_seek_chat_model,
    temperature=0.7,
    max_tokens=1000,
    request_timeout=200
)

message = [HumanMessage(content="请用中文回答：你是谁？")]

response = llm.invoke(message)

print(response.content)

我是DeepSeek Chat，由深度求索公司（DeepSeek）创造的智能AI助手！✨  

我可以回答你的问题、帮你学习、提供建议、陪你聊天，甚至处理各种文本和文件。无论是日常疑问、专业问题，还是写作灵感，我都会尽力帮你！😊  

有什么我可以帮你的吗？


### 1.2-方式2：deeoseek模块

In [10]:
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(model=deep_seek_chat_model, api_key=API_KEY)

message = [HumanMessage(content="讲一个笑话？")]
response = llm.invoke(message)
print(response.content)

好的，来一个程序员专属冷笑话：

**为什么程序员总在夏天迷路？**  

因为出门就进入了“无码（无码=无马路）”状态，还总是遇到“递归式绕圈”——走两步就觉得自己在循环里，最后只能靠“清除缓存”（擦汗）重启导航系统！  

（🤖 温馨提示：本笑话最佳效果需配合空调使用，以防降温过度。）


### 1.3-方式3：使用langchain的统一api结构 init_chat_model

In [12]:
from  langchain.chat_models import init_chat_model

llm3 = init_chat_model(deep_seek_chat_model, api_key=API_KEY, base_url=BASE_URL)

message = [HumanMessage(content="讲一个笑话？")]
response = llm3.invoke(message)
print(response.content)


好的，来一个程序员专属冷笑话：

**为什么程序员总在夏天分不清圣诞节和万圣节？**

因为…… Dec 25（圣诞节）和 Oct 31（万圣节）在他们眼里都是——  
**"25 == 31"** （返回 `false`）😂  

（注：Dec是12月，但程序员会误读成十进制；Oct是8月，但程序员会当成八进制。25在十进制确实是25，而31在八进制等于十进制的……25！所以程序员眼里这俩节日“相等”了。）


### 总结：
1. 统一的用 init_chat_model / ChatXXXModel 初始化一个 chat model
2. 统一的用invoke(messages) or invoke({'xx':'xx'})
3. 我们可以用PromptTemplate构建结构化的模板
4. （不重点），结构化输出

## 2. 了解LangChain Chain

### 2.1 chat agent using langchain

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant."),
        ("human", "帮我写一个和{topic}相关的笑话。"),
    ]
)

prompt_template.invoke({"topic": "Python编程"}).to_messages()

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='帮我写一个和Python编程相关的笑话。', additional_kwargs={}, response_metadata={})]

In [15]:
basic_chain = prompt_template | llm3

response = basic_chain.invoke({"topic": "Python编程"})

print(response.content)


当然！这是一个关于 Python 编程的笑话：

---

**笑话标题：** Python 的无限循环  

程序员 A： "我写了一个无限循环，停不下来了！"  
程序员 B： "用 `Ctrl + C` 啊！"  
程序员 A： "试过了，但 Python 说那是 'KeyboardInterrupt'，不是 'CtrlInterrupt'！"  

---

希望你喜欢！如果有其他主题或风格的需求，可以告诉我调整~ 😄


### 2.2 结构化输出

In [17]:
from pydantic import BaseModel, Field

class Joke(BaseModel):
    """A simple joke model."""
    setup: str = Field(..., description="The setup of the joke")
    punchline: str = Field(..., description="The punchline of the joke")

joke_chain = prompt_template | llm3.with_structured_output(Joke)

joke_response = joke_chain.invoke({"topic": "Python编程"})

print(joke_response)

setup='Why did the Python programmer prefer snakes over other pets?' punchline='Because they already knew how to handle them!'


## 3. LangGraph Memory

In [67]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph, END

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    """Call the model with the current state."""
    response = llm3.invoke(state["messages"])
    return {"messages": response}

workflow.add_node('call_model', call_model)
workflow.add_edge(START, 'call_model')

memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

In [43]:
config = {"configurable": {"thread_id":"abc_123"}}
query = "Hi, I am shizhong!"

input_messages = [HumanMessage(query)]
output = graph.invoke({"messages": input_messages}, config=config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Hi Shizhong! Nice to meet you. 😊 How can I help you today?


In [44]:
query = "请用中文回答：我是谁？"

input_messages = [HumanMessage(query)]
output = graph.invoke({"messages": input_messages}, config=config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

你是 **shizhong**（“始终”或“时钟”的拼音，也可能是你的名字或昵称）——刚刚你就是这样向我介绍自己的呀！😊  

如果这是个哲学问题（比如“我是谁？”），那我可能会说：你是此刻和我对话的独特存在，一个拥有自己故事、想法和目标的个体～ 需要我帮你探索更多吗？


## 4. Agents

### 4.1 Single Agent(with tool)

In [52]:
from langgraph.prebuilt import create_react_agent

def add_numbers(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

agent = create_react_agent(
    tools=[add_numbers, multiply_numbers],
    model=llm3
)

output = agent.invoke({"messages": [HumanMessage("What is 2 + 3?")], "config": config})

for item in output['messages']:
    item.pretty_print()

================================ Human Message =================================

What is 2 + 3?
================================== Ai Message ==================================
Tool Calls:
  add_numbers (call_0_7c924a7d-eba3-4226-a003-7273d9d7d01b)
 Call ID: call_0_7c924a7d-eba3-4226-a003-7273d9d7d01b
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: add_numbers

5
================================== Ai Message ==================================

The result of 2 + 3 is 5.


### 4.2 DeepResearch

In [61]:
! pip install langchain_community
! pip install -U duckduckgo-search
! pip install -U langchain-tavily

#### 4.2.1 整体流程
单一目标原则
1. planning agent 做规划，要搜索什么东西；
2. search agent，根据plan 调用 Search Tool 检索结果；（可以循环的搜索）
3. report agetn，根据检索结果，生成报告。

In [63]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    tavily_api_key=os.getenv('TAVILY_APE_KEY'),
)

search_tool.invoke("What is the capital of France?")

{'query': 'What is the capital of France?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Paris',
   'title': 'Paris - Wikipedia',
   'content': 'Paris is the capital and largest city of France. With an estimated population of 2,048,472 in January 2025 in an area of more than 105 km2 (41 sq mi),',
   'score': 0.85995835,
   'raw_content': None},
  {'url': 'https://home.adelphi.edu/~ca19535/page%204.html',
   'title': 'Paris facts: the capital of France in history',
   'content': 'Paris, France   ## Paris facts: Paris, the **capital of France** Paris is the **capital of France**, Paris has 2.234 million inhabitants ## Paris facts: Paris history Republic, Paris has a rich 2000 year history. See details of Paris churches, including Notre ## Paris facts: Paris, a world city Paris is a world capital city of shopping french fashion brands. All of this turns Paris into a ## Paris facts: the capital of France in history Before

In [64]:
# planner agent 输出 json / markdown  
# create agent with structured output

plan_prompt = """
你是一名规划师，你的任务是为用户提供一个搜索计划。你要用json格式输出搜索计划，列出不超过五个最相关的搜索词。
请确保每个搜索词都能帮助用户找到相关信息。

输出示例：
{
    "search_items": [
        {"search_term": "Python编程"},
        {"search_term": "机器学习"}
    ]
}

你只需要返回这个json格式的输出，不需要其他任何文字。
"""

plan_agent = create_react_agent(
    tools=[],
    model=llm3,
    prompt=plan_prompt,
)

search_agent = create_react_agent(
    tools=[search_tool],
    model=llm3,
    prompt="你是一名搜索助手，你的任务是根据用户的搜索计划执行搜索。请使用TavilySearch工具进行搜索。"
)

report_agent = create_react_agent(
    tools=[],
    model=llm3,
    prompt="你是一名报告生成器，你的任务是根据搜索结果生成一份报告。请使用json格式输出报告。"
)


In [68]:
from pydantic import BaseModel, Field
from typing import List

class ResearchPlan(BaseModel):
    """Research plan with key questions"""
    search_items: List[str] = Field(
        description="List of 1-3 key research search items to investigate",
        max_items=3
    )

class ResearchState(MessagesState):
    """State for the ReSearch workflow."""
    search_plan: ResearchPlan|None = None


workflow = StateGraph(ResearchState)

# Add nodes and edges with state transformations
workflow.add_node("plan_agent", plan_agent)
workflow.add_node("search_agent", search_agent)
workflow.add_node("report_agent", report_agent)

workflow.add_edge(START, "plan_agent")
workflow.add_edge("plan_agent", "search_agent")
workflow.add_edge("search_agent", "report_agent")
workflow.add_edge("report_agent", END)

# Compile workflow
app = workflow.compile()

In [69]:
output = app.invoke(
    {"messages": [
        HumanMessage("write an description about andrej karthy")
    ]}
)

In [70]:
for item in output["messages"]:
    item.pretty_print()

================================ Human Message =================================

write an description about andrej karthy
================================== Ai Message ==================================

```json
{
    "search_items": [
        {"search_term": "Andrej Karpathy"},
        {"search_term": "Andrej Karpathy OpenAI"},
        {"search_term": "Andrej Karpathy Tesla"},
        {"search_term": "Andrej Karpathy neural networks"},
        {"search_term": "Andrej Karpathy blog"}
    ]
}
```
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_de93509b-884e-46c2-a8e5-4763d394a582)
 Call ID: call_0_de93509b-884e-46c2-a8e5-4763d394a582
  Args:
    query: Andrej Karpathy
  tavily_search (call_1_95a616c4-608f-4a67-a407-0116b64cfca9)
 Call ID: call_1_95a616c4-608f-4a67-a407-0116b64cfca9
  Args:
    query: Andrej Karpathy OpenAI
  tavily_search (call_2_d3f6b65c-00a8-46bc-9825-2882f4957d6d)
 Call ID: call_2_d3f6b65c-00a8-46b